# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,high rock,26.62,-78.28,73.98,76,77,25.17,BS,1589437697
1,1,turukhansk,65.82,87.98,45.73,53,0,7.65,RU,1589437099
2,2,cariati,39.49,16.96,73.00,52,88,11.41,IT,1589437697
3,3,aklavik,68.22,-135.01,20.98,95,99,5.44,CA,1589437118
4,4,rochester,43.15,-77.62,42.01,74,75,6.40,US,1589437484


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,rikitea,-23.12,-134.97,77.74,84,0,9.04,PF,1589437657
290,290,iskenderun,36.59,36.17,72.00,60,0,4.70,TR,1589437747
305,305,shaki,41.08,47.17,73.40,53,0,2.24,AZ,1589437750
319,319,cuamba,-14.80,36.54,73.29,54,0,3.00,MZ,1589437752
400,400,koungheul,13.98,-14.80,75.38,39,0,6.73,SN,1589437766
527,527,bam,29.11,58.36,78.10,15,0,8.68,IR,1589437789


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
21,rikitea,PF,-23.12,-134.97,
290,iskenderun,TR,36.59,36.17,
305,shaki,AZ,41.08,47.17,
319,cuamba,MZ,-14.80,36.54,
400,koungheul,SN,13.98,-14.80,
527,bam,IR,29.11,58.36,


In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
21,rikitea,PF,-23.12,-134.97,Pension Maro'i
290,iskenderun,TR,36.59,36.17,Grand Akçalı Otel
305,shaki,AZ,41.08,47.17,Hotel Sheki Azpetrol
319,cuamba,MZ,-14.80,36.54,Hotel Vision
400,koungheul,SN,13.98,-14.80,
527,bam,IR,29.11,58.36,Bam Tourist Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))